In [5]:
#import experiments
import SeqDHBM
import fasta
import sys

if (__name__ == "__main__"):
    fastafile = "test.fasta"
    for params in sys.argv:
        if params.startswith("-fasta="):
            fastafile = params[7:]
    
    fasta.fasta_to_seq()
    fo = "Saved_run/IL-1A(271)/IL-1A(271).fasta"
    SeqDHBM.ReadFasta(fo)
    #SeqDHBM.SpotCoordinationSite(SeqDHBM.ReadFasta(fo), "structure")


TypeError: fasta_to_seq() missing 1 required positional argument: 'in_file'